## Scraping Waterparks Lyrics

In [75]:
import pandas as pd
import numpy as np
import json
import spotipy
import re
from lyricsgenius import Genius
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
API_KEY = "sFB433-aPPAH8VpsfF9ETSr-9SQYsbxFBGMTd_XvqF6ED84O-fMYiGf_ENUtNLRn"

In [3]:
genius = Genius(API_KEY)
artist = genius.search_artist('Waterparks')

Searching for songs by Waterparks...

Song 1: "Turbulent"
Song 2: "I Miss Having Sex But at Least I Don’t Wanna Die Anymore"
Song 3: "Stupid for You"
Song 4: "TANTRUM"
Song 5: "21 Questions"
Song 6: "Blonde"
Song 7: "Lucky People"
Song 8: "Watch What Happens Next"
Song 9: "We Need to Talk"
Song 10: "Not Warriors"
Song 11: "11:11"
Song 12: "Snow Globe"
Song 13: "Gloom Boys"
Song 14: "High Definition"
Song 15: "Peach (Lobotomy)"
Song 16: "Numb"
Song 17: "Royal"
Song 18: "Dream Boy"
Song 19: "Crave"
Song 20: "Lowkey as Hell"
Song 21: "Rare"
Song 22: "Telephone"
Song 23: "[Reboot]"
Song 24: "War Crimes"
Song 25: "You’d Be Paranoid Too (If Everyone Was Out to Get You)"
Song 26: "Dizzy"
Song 27: "Violet!"
Song 28: "FUNERAL GREY"
Song 29: "Easy To Hate"
Song 30: "See You In The Future"
Song 31: "I Felt Younger When We Met"
Song 32: "Crybaby"
Song 33: "Plum Island"
Song 34: "Take Her to the Moon"
Song 35: "Sleep Alone"
Song 36: "Hawaii (Stay Awake)"
Song 37: "BRAINWASHED"
Song 38: "REAL SUPER 

Writing all lyrics to a JSON file, which contains metadata for each song.

In [5]:
artist.save_lyrics(filename="all_waterparks_lyrics.json", extension="json")

Wrote all_waterparks_lyrics.json.


In [8]:
# load in the json file containing song info
json_file = open('all_waterparks_lyrics.json')
data = json.load(json_file)

In [9]:
# put song title, artists, release date, and lyrics into a dataframe
song_dicts = []
for song in data['songs']:
    song_title = song["title"]
    artist = song["artist_names"]
    release_date = song["release_date"]
    lyrics = song["lyrics"]
    song_dict = {"song_title": song_title, 
                 "artist": artist, 
                 "release_date": release_date, 
                 "lyrics": lyrics}
    song_dicts.append(song_dict)
 
json_file.close()
track_info = pd.DataFrame(song_dicts)
track_info

,song_title,artist,release_date,lyrics
0,Turbulent,Waterparks,2019-05-23,56 ContributorsTurbulent Lyrics[Verse 1]\nYou ...
1,I Miss Having Sex But at Least I Don’t Wanna D...,Waterparks,2019-10-11,44 ContributorsTranslationsPolskiI Miss Having...
2,Stupid for You,Waterparks,2016-08-31,48 ContributorsStupid for You Lyrics[Chorus]\n...
3,TANTRUM,Waterparks,2018-01-26,59 ContributorsTANTRUM Lyrics[Intro]\nFuck the...
4,21 Questions,Waterparks,2016-11-04,32 Contributors21 Questions Lyrics[Verse 1]\nI...
...,...,...,...,...
131,LOVE ABOUT IT,Waterparks (Ft. ​blackbear),2022-10-13,3 ContributorsLOVE ABOUT IT Lyrics[Verse 1: Aw...
132,Lowkey As Heck,Waterparks,2020-09-24,2 ContributorsLowkey As Heck Lyrics[Verse 1]\n...
133,Violet! (MC4D Remix),Waterparks,2021-12-06,2 ContributorsViolet! (MC4D Remix) Lyrics[Vers...
134,Dream Boy (MC4D Remix),Waterparks,2020-03-19,3 ContributorsDream Boy (MC4D Remix) Lyrics[Ve...


In [10]:
np.unique(track_info["song_title"])

array(['11:11', '2 BEST FRIENDS', '21 Questions', '60 minutes (DEMO)',
       'A NIGHT OUT ON EARTH', 'All By Myself', 'American Graffiti',
       'American History', 'BRAINWASHED', 'Beating Heart Baby', 'Beep Me',
       'Blonde', 'Bones of ’92', 'CLOSER', 'Candy', 'Cherry Red',
       'Christmas', 'Cobblestone Grey (Funeral Grey Parody)', 'Crave',
       'Crybaby', 'Crying Over It All', 'Dizzy', 'Double Dare 2019',
       'Dream Boy',
       'Dream Boy (Acoustic In The Bathroom At Awsten’s Apartment Version)',
       'Dream Boy (JVNA Remix)', 'Dream Boy (MC4D Remix)',
       'END OF THE WATER (Alt Version)', 'END OF THE WATER (FEEL)',
       'Easter Egg', 'Easy To Hate', 'Entertainment 2019', 'Everywhere',
       'FUCK ABOUT IT', 'FUNERAL GREY', 'Fantastic', 'Fantastic (Demo)',
       'Friendly Reminder', 'Fruit Roll Ups', 'Fuzzy',
       'Gladiator (Interlude)', 'Glitter Times (1997 DEMO)', 'Gloom Boys',
       'Greatest Hits', 'Group Chat', 'Gwen Stefani',
       'Hawaii (Stay Awak

Notice from the above that there are lots of remixes and demos etc. that I ultimately do not want included in the game. So, I created a playlist of the songs that I *do* want included in Spotify and I will scrape the titles from there to filter unwanted songs out. 

I've also created a smaller playlist that consists of only songs from EPs and albums (no demos, covers, etc.).

In [12]:
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id="e26bdc1cd33b4f56b034cbe51a345704", 
                                                                              client_secret="a3afcb8abca942b9b94c8e776fd8e5c6"))

In [18]:
all_playlist = "https://open.spotify.com/playlist/3vv4oGMDgMPTJ5BCoUxxr9?si=01d8be4dc3c94632"
subset_playlist = "https://open.spotify.com/playlist/1diqTEL8WlPNxEAHfbgUGs?si=f679c7f972d14e87"

In [15]:
# function that retrieves all track names from playlist
def get_track_names(playlist_link, spotify=spotify):
    track_names = []
    for track_info in spotify.playlist_tracks(playlist_link)["items"]: 
        info = track_info["track"]
        track_name = info["name"]
        track_names.append(track_name)
    return track_names

In [55]:
all_songs = get_track_names(all_playlist)
print(len(all_songs))
print(all_songs)

97
['Silver', "Bones of '92", 'I Was Hiding Under Your Porch Because I Love You', 'They All Float', 'Fantastic', 'Christmas (feat. Tyler Welsh)', 'American History', 'Easter Egg', 'New Wave', 'Night Maps (feat. Fred Mascherino)', "I'm a Natural Blue", 'Crave', 'Territory', 'Mad All the Time', 'Pink', 'No Capes', 'Hawaii (Stay Awake)', 'Gloom Boys', 'Stupid for You', 'Royal', 'Take Her to the Moon', 'Made in America', 'Dizzy', 'Powerless', 'Little Violence', '21 Questions', 'It Follows', 'Plum Island', "I'll Always Be Around", '11:11', 'Blonde', 'Peach (Lobotomy)', 'We Need to Talk', 'Not Warriors', 'Lucky People', 'Rare', 'TANTRUM', 'Crybaby', 'Sleep Alone', 'rainy days in la', 'Beating Heart Baby', 'Cherry Red', 'Watch What Happens Next', 'Dream Boy', 'Easy To Hate', 'High Definition', 'Telephone', 'Group Chat', 'Turbulent', 'Never Bloom Again', "I Miss Having Sex But At Least I Don't Wanna Die Anymore", 'War Crimes', '[Reboot]', 'Worst', 'Zone Out', 'I Felt Younger When We Met', 'Gre

In [48]:
album_songs = get_track_names(subset_playlist)
print(len(album_songs))
print(album_songs)

83
['Silver', "Bones of '92", 'I Was Hiding Under Your Porch Because I Love You', 'They All Float', 'Fantastic', 'Christmas (feat. Tyler Welsh)', 'American History', 'Easter Egg', 'New Wave', 'Night Maps (feat. Fred Mascherino)', "I'm a Natural Blue", 'Crave', 'Territory', 'Mad All the Time', 'Pink', 'No Capes', 'Hawaii (Stay Awake)', 'Gloom Boys', 'Stupid for You', 'Royal', 'Take Her to the Moon', 'Made in America', 'Dizzy', 'Powerless', 'Little Violence', '21 Questions', 'It Follows', 'Plum Island', "I'll Always Be Around", '11:11', 'Blonde', 'Peach (Lobotomy)', 'We Need to Talk', 'Not Warriors', 'Lucky People', 'Rare', 'TANTRUM', 'Crybaby', 'Sleep Alone', 'Cherry Red', 'Watch What Happens Next', 'Dream Boy', 'Easy To Hate', 'High Definition', 'Telephone', 'Group Chat', 'Turbulent', 'Never Bloom Again', "I Miss Having Sex But At Least I Don't Wanna Die Anymore", 'War Crimes', '[Reboot]', 'Worst', 'Zone Out', 'I Felt Younger When We Met', 'Greatest Hits', 'Fuzzy', 'Lowkey As Hell', 'N

We need to correspond the scraped Genius lyrics to the song titles from the Spotify playlists. 

In [25]:
# songs I want to include in all_songs but do not have corresponding lyrics
set([song.lower() for song in all_songs]) - set(track_info["song_title"].str.lower())

{"bones of '92",
 'christmas (feat. tyler welsh)',
 'fuck about it (feat. blackbear)',
 "i miss having sex but at least i don't wanna die anymore",
 "i'll always be around",
 "i'm a natural blue",
 'night maps (feat. fred mascherino)',
 'rainy days in la',
 'see you in in the future',
 "sorry as i'll ever be (1997 demo)",
 "why can't i"}

In [26]:
set([song.lower() for song in album_songs]) - set(track_info["song_title"].str.lower())

{"bones of '92",
 'christmas (feat. tyler welsh)',
 'fuck about it (feat. blackbear)',
 "i miss having sex but at least i don't wanna die anymore",
 "i'll always be around",
 "i'm a natural blue",
 'night maps (feat. fred mascherino)',
 'see you in in the future'}

Reasons for the missing correspondances include: 

- Mismatched apostrophes
- Parentheses, e.g. (feat. blackbear)
- See You In *In* The Future
- Question mark for Why Can't I?
- Rainy days in LA not found

In [56]:
# fixing the names in the dataframe
## apostrophe
track_info["song_title"] = track_info["song_title"].str.replace("’", "'")
## add the parentheses (for matching purposes, later on all parentheses will be removed)
track_info.loc[track_info["song_title"] == "FUCK ABOUT IT", "song_title"] = "FUCK ABOUT IT (feat. blackbear)"
track_info.loc[track_info["song_title"] == "Christmas", "song_title"] = "Christmas (feat. Tyler Welsh)"
track_info.loc[track_info["song_title"] == "Night Maps", "song_title"] = "Night Maps (feat. Fred Mascherino)"
# see you in the future
track_info.loc[track_info["song_title"] == "See You In The Future", "song_title"] = "See You In In The Future"
# correcting Why Can't I? on my end
all_songs.remove("Why Can't I")
all_songs = all_songs + ["WHY CAN'T I?"]
# removing rainy days in LA
all_songs.remove("rainy days in la")

In [57]:
len(all_songs)

96

In [58]:
set([song.lower() for song in all_songs]) - set(track_info["song_title"].str.lower())

{'you’d be paranoid too (if everyone was out to get you)'}

In [59]:
set([song.lower() for song in album_songs]) - set(track_info["song_title"].str.lower())

{'you’d be paranoid too (if everyone was out to get you)'}

In [60]:
# some weird apostrophe inconsistencies - will fix here
all_songs.remove("You’d Be Paranoid Too (If Everyone Was Out To Get You)")
all_songs = all_songs + ["You'd Be Paranoid Too (If Everyone Was Out to Get You)"]

In [61]:
album_songs.remove("You’d Be Paranoid Too (If Everyone Was Out To Get You)")
album_songs = album_songs + ["You'd Be Paranoid Too (If Everyone Was Out to Get You)"]

In [62]:
set([song.lower() for song in all_songs]) - set(track_info["song_title"].str.lower())

set()

In [63]:
set([song.lower() for song in album_songs]) - set(track_info["song_title"].str.lower())

set()

Track title inconsistencies all fixed!

The next step is to remove all songs from the dataframe who are not in all_songs.

In [103]:
track_info_filt = track_info[track_info['song_title'].str.lower().isin([song.lower() for song in all_songs])]

In [104]:
track_info_filt

,song_title,artist,release_date,lyrics
0,Turbulent,Waterparks,2019-05-23,56 ContributorsTurbulent Lyrics[Verse 1]\nYou ...
1,I Miss Having Sex But at Least I Don't Wanna D...,Waterparks,2019-10-11,44 ContributorsTranslationsPolskiI Miss Having...
2,Stupid for You,Waterparks,2016-08-31,48 ContributorsStupid for You Lyrics[Chorus]\n...
3,TANTRUM,Waterparks,2018-01-26,59 ContributorsTANTRUM Lyrics[Intro]\nFuck the...
4,21 Questions,Waterparks,2016-11-04,32 Contributors21 Questions Lyrics[Verse 1]\nI...
...,...,...,...,...
93,Bones of '92,Waterparks,2012-04-03,14 ContributorsBones of ’92 Lyrics[Chorus]\nSi...
94,New Wave,Waterparks,2014-06-04,8 ContributorsNew Wave Lyrics[Verse 1]\nCall i...
96,WHY CAN'T I?,Waterparks,2023-04-14,"6 ContributorsWHY CAN’T I? Lyrics[Verse 1]\n""G..."
99,"LIFE IS PUKE (20,000) (1995 DEMO)",Waterparks,2020-01-22,"12 ContributorsLIFE IS PUKE (20,000) (1995 DEM..."


And now we have to clean the lyrics.

Notice that each of the lyrics strings are structured as below: 

In [105]:
track_info_filt["lyrics"][56]

"29 ContributorsST*RFUCKER Lyrics[Intro]\n(Yo, imagine an album starting with this)\n(Like a big breath, close your eyes and shit)\n\n[Verse 1]\nI'm gonna move out of my loft\nAnd into a limousine\nSo I can drive you fuckin' crazy\nAnd crash out where I sleep, yeah\nJesus Christ won't text me back\nI'll always be around, in fact\nI'm climbin' up to your window now\nRunnin' back to you at the speed of sound\n\n[Chorus]\nIt's been a pleasure\nIt's nice to meet you\nIt's been a pleasure\nIt's nice to meet you\nIt's been a pleasure (So destroy my world, I'll tape the cracks)\nIt's nice to meet you ('Cause what you give out's what you get back)\nIt's been a pleasure (I'll stop the world, I'll tape the cracks)\nIt's nice to meet you (What you give out's what you get back)\n\n[Verse 2]\nMaybe I'm a soul-sucker\nBut you're just a starfucker\nI signed somebody's NDA\nI'll go away, but LA's such a fuckin' weird place (One, two, three, four)\nBet you wouldn't ask me to do that by this time next y

- There is a bunch of junk up to [Verse 1] or [Intro] (the first "[")
- At the very end, there is some number + Embed

Other things that need to be fixed: 
- Unicode characters (e.g. \u2005a) should be replaced with a space
- Newline \n. Will keep for now to split each song's lyrics into lines
- Section indicators, e.g. [Chorus]. Will keep for now to create a new column with the section

In [106]:
# remove everything up to the first [
track_info_filt["lyrics"] = track_info_filt["lyrics"].replace(r"^.*?\[", "[", regex=True)

/tmp/ipykernel_24467/4171974222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  track_info_filt["lyrics"] = track_info_filt["lyrics"].replace(r"^.*?\[", "[", regex=True)


In [107]:
track_info_filt["lyrics"][56]

"[Intro]\n(Yo, imagine an album starting with this)\n(Like a big breath, close your eyes and shit)\n\n[Verse 1]\nI'm gonna move out of my loft\nAnd into a limousine\nSo I can drive you fuckin' crazy\nAnd crash out where I sleep, yeah\nJesus Christ won't text me back\nI'll always be around, in fact\nI'm climbin' up to your window now\nRunnin' back to you at the speed of sound\n\n[Chorus]\nIt's been a pleasure\nIt's nice to meet you\nIt's been a pleasure\nIt's nice to meet you\nIt's been a pleasure (So destroy my world, I'll tape the cracks)\nIt's nice to meet you ('Cause what you give out's what you get back)\nIt's been a pleasure (I'll stop the world, I'll tape the cracks)\nIt's nice to meet you (What you give out's what you get back)\n\n[Verse 2]\nMaybe I'm a soul-sucker\nBut you're just a starfucker\nI signed somebody's NDA\nI'll go away, but LA's such a fuckin' weird place (One, two, three, four)\nBet you wouldn't ask me to do that by this time next year\nYou can't hide my face but 

In [108]:
# removing the nEmbed or Embed at the end of the string
track_info_filt["lyrics"] = track_info_filt["lyrics"].replace(r"\d*Embed\'*\"*$", "", regex=True)

/tmp/ipykernel_24467/990332170.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  track_info_filt["lyrics"] = track_info_filt["lyrics"].replace(r"\d*Embed\'*\"*$", "", regex=True)


In [109]:
track_info_filt["lyrics"][56]

"[Intro]\n(Yo, imagine an album starting with this)\n(Like a big breath, close your eyes and shit)\n\n[Verse 1]\nI'm gonna move out of my loft\nAnd into a limousine\nSo I can drive you fuckin' crazy\nAnd crash out where I sleep, yeah\nJesus Christ won't text me back\nI'll always be around, in fact\nI'm climbin' up to your window now\nRunnin' back to you at the speed of sound\n\n[Chorus]\nIt's been a pleasure\nIt's nice to meet you\nIt's been a pleasure\nIt's nice to meet you\nIt's been a pleasure (So destroy my world, I'll tape the cracks)\nIt's nice to meet you ('Cause what you give out's what you get back)\nIt's been a pleasure (I'll stop the world, I'll tape the cracks)\nIt's nice to meet you (What you give out's what you get back)\n\n[Verse 2]\nMaybe I'm a soul-sucker\nBut you're just a starfucker\nI signed somebody's NDA\nI'll go away, but LA's such a fuckin' weird place (One, two, three, four)\nBet you wouldn't ask me to do that by this time next year\nYou can't hide my face but 

In [110]:
# prior to removing unicode
track_info_filt["lyrics"][21]

"[Verse 1]\nI'd like to know your middle name\nLet's talk about your parents and your future dreams\nI'm\u2005interested\u2005but distant to\u2005a fault\nAnd I'd never want to\u2005complicate your heart\n\n[Pre-Chorus]\nI've gotta let you know\nThat I think that I love you so\nYou could be my only one\n'Cause I think that I love you so\n\n[Chorus]\nI know we only just met\nSo why do I feel invested?\nDo you feel it too?\nDo you feel it too?\nI can be your best yet\nFuture favorite regret\nDo you feel it too?\nDo you feel it too?\n\n[Post-Chorus]\nSo I'll just tell my telephone\nAll of the above when I'm alone\nYou might also like[Verse 2]\nI talk a lot, but we could fill your frames\nWith pictures of our faces 'til we share a name\nI'm living on a target and you shot it with an arrow\nNow I lost my self-control, I can't stop thinking\nAnd I'm thinking that I lost it\nAll these aisles feel like miles\nWhere you go, I'll follow\n\n[Chorus]\nI know we only just met\nSo why do I feel inve

In [111]:
# remove unicode character \u2005 and replace w space
track_info_filt["lyrics"] = track_info_filt["lyrics"].replace(r"\u2005", " ", regex=True)

/tmp/ipykernel_24467/1388096566.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  track_info_filt["lyrics"] = track_info_filt["lyrics"].replace(r"\u2005", " ", regex=True)


In [112]:
track_info_filt["lyrics"][21]

"[Verse 1]\nI'd like to know your middle name\nLet's talk about your parents and your future dreams\nI'm interested but distant to a fault\nAnd I'd never want to complicate your heart\n\n[Pre-Chorus]\nI've gotta let you know\nThat I think that I love you so\nYou could be my only one\n'Cause I think that I love you so\n\n[Chorus]\nI know we only just met\nSo why do I feel invested?\nDo you feel it too?\nDo you feel it too?\nI can be your best yet\nFuture favorite regret\nDo you feel it too?\nDo you feel it too?\n\n[Post-Chorus]\nSo I'll just tell my telephone\nAll of the above when I'm alone\nYou might also like[Verse 2]\nI talk a lot, but we could fill your frames\nWith pictures of our faces 'til we share a name\nI'm living on a target and you shot it with an arrow\nNow I lost my self-control, I can't stop thinking\nAnd I'm thinking that I lost it\nAll these aisles feel like miles\nWhere you go, I'll follow\n\n[Chorus]\nI know we only just met\nSo why do I feel invested?\nDo you feel i

Now, we will expand the string of lyrics for each song into individual lines (split by \n), and we will also designate song sections to each line. 

In [158]:
track_info_filt

,song_title,artist,release_date,lyrics
0,Turbulent,Waterparks,2019-05-23,[Verse 1]\nYou had your own Pete Wentz and Pat...
1,I Miss Having Sex But at Least I Don't Wanna D...,Waterparks,2019-10-11,"[Verse 1]\nI'm sick of all this, ""How'd you ge..."
2,Stupid for You,Waterparks,2016-08-31,"[Chorus]\nHey, tell me what you want me to say..."
3,TANTRUM,Waterparks,2018-01-26,[Intro]\nFuck these fuckboy bands that can't t...
4,21 Questions,Waterparks,2016-11-04,[Verse 1]\nI wish there was a situation to be ...
...,...,...,...,...
93,Bones of '92,Waterparks,2012-04-03,[Chorus]\nSimple steps are all it takes\nBut s...
94,New Wave,Waterparks,2014-06-04,[Verse 1]\nCall it new wave\nSay it like you s...
96,WHY CAN'T I?,Waterparks,2023-04-14,"[Verse 1]\n""Get a load of me?"" Get a load of y..."
99,"LIFE IS PUKE (20,000) (1995 DEMO)",Waterparks,2020-01-22,"[Spoken]\nOkay, so, you guys are assholes\nI d..."


In [234]:
# takes in a row of the df (representing a single song) and splits it into several rows using section
def split_by_section(song_title, artist, lyrics): 
    newstr = lyrics
    sections = []
    while True: 
        try: 
            # find the section label, e.g. [Verse 1]
            match = re.search(r"\[.*?\]", newstr)
            section = match.group(0)
            # split the entire string based on that label (one split only)
            newstr = newstr.split(section, 1)[1]

            # find the next section
            next_match = re.search(r"\[.*?\]", newstr)
            # if there is no match, next_match will be None
            if next_match:
                section_lyrics = newstr.split(next_match.group(0), 1)[0]
            else:
                section_lyrics = newstr

            sections.append({
                'song_title': song_title,
                'artist': artist,
                'section': section,
                'section_lyrics': section_lyrics
            })
        except AttributeError:
            break
    
    return sections

In [235]:
all_sections = []
for index, row in track_info_filt.iterrows():
    song_sections = split_by_section(row['song_title'], row["artist"], row["lyrics"])
    all_sections.append(song_sections)

In [238]:
all_sections_flat = [x for xs in all_sections for x in xs]
lyrics_sections_df = pd.DataFrame(all_sections_flat)
lyrics_sections_df

,song_title,artist,section,section_lyrics
0,Turbulent,Waterparks,[Verse 1],\nYou had your own Pete Wentz and Patrick comb...
1,Turbulent,Waterparks,[Pre-Chorus],\nSo fuck yourself and fuck your feelings\nI b...
2,Turbulent,Waterparks,[Chorus],"\nI'd unfuck you if I could, ooh, ooh, ooh\nI'..."
3,Turbulent,Waterparks,[Verse 2],\nYou had your own Awsten Knight (Hi)\nHe text...
4,Turbulent,Waterparks,[Pre-Chorus],\nSo fuck yourself and fuck your feelings\nI b...
...,...,...,...,...
743,Beating Heart Baby,Waterparks,[Intro],"\nBaby, is this love for real?\nLet me in your..."
744,Beating Heart Baby,Waterparks,[Verse 1],"\nYou, you want nothing to do with me\nCause y..."
745,Beating Heart Baby,Waterparks,[Chorus],\nBaby is this love for real?\nLet me in your ...
746,Beating Heart Baby,Waterparks,[Verse 2],"\nGirl, you really got your hold on me\nCause ..."


In [239]:
lyrics_sections_df.to_csv("Lyrics_sections.csv", index=False)

In [240]:
lyrics_sections_df["section_lyrics"].loc[0]

'\nYou had your own Pete Wentz and Patrick combined\nWriting you songs and singing them all like\nEvery single day\nTour brings that special pain\n\n'

In [241]:
full_lyrics = lyrics_sections_df.assign(lyrics=lyrics_sections_df["section_lyrics"].str.split("\n")).explode("lyrics").drop(columns=["section_lyrics"])

In [242]:
full_lyrics

,song_title,artist,section,lyrics
0,Turbulent,Waterparks,[Verse 1],
0,Turbulent,Waterparks,[Verse 1],You had your own Pete Wentz and Patrick combined
0,Turbulent,Waterparks,[Verse 1],Writing you songs and singing them all like
0,Turbulent,Waterparks,[Verse 1],Every single day
0,Turbulent,Waterparks,[Verse 1],Tour brings that special pain
...,...,...,...,...
747,Beating Heart Baby,Waterparks,[Chorus],"Your beating heart, baby, baby"
747,Beating Heart Baby,Waterparks,[Chorus],So baby is this love for real?
747,Beating Heart Baby,Waterparks,[Chorus],"Let me in your arms to feel, woah"
747,Beating Heart Baby,Waterparks,[Chorus],"Your beating heart, baby"


In [244]:
full_lyrics = full_lyrics[["song_title", "artist", "section", "lyrics"]]

In [248]:
full_lyrics = full_lyrics[full_lyrics['lyrics'].str.strip() != '']

In [249]:
full_lyrics

,song_title,artist,section,lyrics
1,Turbulent,Waterparks,[Verse 1],You had your own Pete Wentz and Patrick combined
2,Turbulent,Waterparks,[Verse 1],Writing you songs and singing them all like
3,Turbulent,Waterparks,[Verse 1],Every single day
4,Turbulent,Waterparks,[Verse 1],Tour brings that special pain
8,Turbulent,Waterparks,[Pre-Chorus],So fuck yourself and fuck your feelings
...,...,...,...,...
6993,Beating Heart Baby,Waterparks,[Chorus],"Your beating heart, baby, baby"
6994,Beating Heart Baby,Waterparks,[Chorus],So baby is this love for real?
6995,Beating Heart Baby,Waterparks,[Chorus],"Let me in your arms to feel, woah"
6996,Beating Heart Baby,Waterparks,[Chorus],"Your beating heart, baby"


In [253]:
full_lyrics['section'] = full_lyrics['section'].str.strip('[]')

/tmp/ipykernel_24467/3806218361.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_lyrics['section'] = full_lyrics['section'].str.strip('[]')


In [254]:
full_lyrics

,song_title,artist,section,lyrics
1,Turbulent,Waterparks,Verse 1,You had your own Pete Wentz and Patrick combined
2,Turbulent,Waterparks,Verse 1,Writing you songs and singing them all like
3,Turbulent,Waterparks,Verse 1,Every single day
4,Turbulent,Waterparks,Verse 1,Tour brings that special pain
8,Turbulent,Waterparks,Pre-Chorus,So fuck yourself and fuck your feelings
...,...,...,...,...
6993,Beating Heart Baby,Waterparks,Chorus,"Your beating heart, baby, baby"
6994,Beating Heart Baby,Waterparks,Chorus,So baby is this love for real?
6995,Beating Heart Baby,Waterparks,Chorus,"Let me in your arms to feel, woah"
6996,Beating Heart Baby,Waterparks,Chorus,"Your beating heart, baby"


In [255]:
full_lyrics.to_csv("full_lyrics.csv", index=False)

I manually added the albums to each song.

In [268]:
all_lyrics = pd.read_csv("full_lyrics_with_albums.csv")
all_lyrics

,song_title,artist,section,lyrics,album
0,Turbulent,Waterparks,Verse 1,You had your own Pete Wentz and Patrick combined,FANDOM
1,Turbulent,Waterparks,Verse 1,Writing you songs and singing them all like,FANDOM
2,Turbulent,Waterparks,Verse 1,Every single day,FANDOM
3,Turbulent,Waterparks,Verse 1,Tour brings that special pain,FANDOM
4,Turbulent,Waterparks,Pre-Chorus,So fuck yourself and fuck your feelings,FANDOM
...,...,...,...,...,...
5156,Beating Heart Baby,Waterparks,Chorus,"Your beating heart, baby, baby",Beating Heart Baby
5157,Beating Heart Baby,Waterparks,Chorus,So baby is this love for real?,Beating Heart Baby
5158,Beating Heart Baby,Waterparks,Chorus,"Let me in your arms to feel, woah",Beating Heart Baby
5159,Beating Heart Baby,Waterparks,Chorus,"Your beating heart, baby",Beating Heart Baby


In [269]:
# checking that there are no typos
np.unique(all_lyrics['album'])

array(['1 (A COLLECTION OF UNRELEASED HOME DEMOS, THIS IS NOT G, OR EVEN AN ALBUM, SHUT UP ENJOY)',
       'Airplane Conversations', 'Beating Heart Baby', 'Black Light',
       'Cluster', 'Double Dare', 'Double Dare (Bonus Tracks)',
       'Entertainment', 'FANDOM', 'Greatest Hits',
       'INTELLECTUAL PROPERTY',
       'INTELLECTUAL PROPERTY (Target Exclusive)'], dtype=object)

Scraping from Genius resulted in this issue where it'd use "You might also like" as a lyric. I'm going to go ahead and remove those. 

In [270]:
all_lyrics = all_lyrics[all_lyrics['lyrics'] != "You might also like"]
all_lyrics

,song_title,artist,section,lyrics,album
0,Turbulent,Waterparks,Verse 1,You had your own Pete Wentz and Patrick combined,FANDOM
1,Turbulent,Waterparks,Verse 1,Writing you songs and singing them all like,FANDOM
2,Turbulent,Waterparks,Verse 1,Every single day,FANDOM
3,Turbulent,Waterparks,Verse 1,Tour brings that special pain,FANDOM
4,Turbulent,Waterparks,Pre-Chorus,So fuck yourself and fuck your feelings,FANDOM
...,...,...,...,...,...
5156,Beating Heart Baby,Waterparks,Chorus,"Your beating heart, baby, baby",Beating Heart Baby
5157,Beating Heart Baby,Waterparks,Chorus,So baby is this love for real?,Beating Heart Baby
5158,Beating Heart Baby,Waterparks,Chorus,"Let me in your arms to feel, woah",Beating Heart Baby
5159,Beating Heart Baby,Waterparks,Chorus,"Your beating heart, baby",Beating Heart Baby


In [271]:
full_lyrics.to_csv("WATERPARKS_LYRICS_FULL.csv", index=False)